In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [17]:
# 한글 폰트 오류 해결
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname = font_path).get_name()
rc("font", family = font_name)

# 데이터 불러오기 및 데이터 이해하기

- 데이터 분석 목표를 정하기 전, 데이터에 대한 사전 지식 및 이해가 필요하다고 판단

In [5]:
df = pd.read_csv("./data/coffee.csv")
df.head()

,Submission ID,Respondent ID,Submitted at,What is your age?,What is your ZIP code?,How many cups of coffee do you typically drink per day?,Where do you typically drink coffee?,Where do you typically drink coffee? (At home),Where do you typically drink coffee? (At the office),Where do you typically drink coffee? (On the go),...,Gender,Gender (please specify),Marital Status,Household Income,Education Level,Ethnicity/Race,Ethnicity/Race (please specify),Employment Status,Number of Children,Political Affiliation
0,gMR29l,vDzkBA,9/11/23 20:50,18-24 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BkPN0e,9Z4VME,9/11/23 20:52,25-34 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,W5G8jj,XoNLQj,9/12/23 1:33,25-34 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4xWgGr,Gp7gde,9/12/23 12:58,35-44 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,QD27Q8,Bp9ogQ,9/12/23 15:24,25-34 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
pd.set_option("display.max_rows", 120)

In [10]:
df.isnull().sum()

Submission ID                                                                                          0
Respondent ID                                                                                          0
Submitted at                                                                                           0
What is your age?                                                                                     31
What is your ZIP code?                                                                                78
How many cups of coffee do you typically drink per day?                                               93
Where do you typically drink coffee?                                                                  70
Where do you typically drink coffee? (At home)                                                        67
Where do you typically drink coffee? (At the office)                                                  67
Where do you typically drink coffee? (On the go)       

- 총 118개의 컬럼이 있음
  - Boolean : 52개
  - String : 44개
  - Interger : 13개
  - Other : 9개

# 분석 목표 설정하기

- 나이, 성별 및 소득 수준에 따른 선호하는 커피에 대한 관계 조사하기

# EDA

## 사용할 컬럼 확인하기

In [12]:
# 나이대별 확인하기
df["What is your age?"].value_counts()

What is your age?
25-34 years old    1986
35-44 years old     960
18-24 years old     461
45-54 years old     302
55-64 years old     187
>65 years old        95
<18 years old        20
Name: count, dtype: int64

- 총 4011명 중에서 약 74% 정도가 25~44세에 해당함을 알 수 있음
  - 나머지 나이를 다 합쳐도 약 26%정도
- 만약 커피를 한다면 이 나이대를 타게팅으로 하는게 좋아보임
  - 즉, 이 74%의 핵심 고객층들이 어떠한 커피를 좋아하는지 아는게 중요함

In [16]:
prefer_coffee = df["What is your favorite coffee drink?"].value_counts()
print(prefer_coffee)2

What is your favorite coffee drink?
Pourover                            1084
Latte                                680
Regular drip coffee                  442
Cappuccino                           341
Espresso                             330
Cortado                              312
Americano                            249
Iced coffee                          156
Mocha                                118
Other                                114
Cold brew                            109
Blended drink (e.g. Frappuccino)      45
Name: count, dtype: int64


In [30]:
# 성별 확인하기
df["Gender"].value_counts()

Gender
Male                      2524
Female                     853
Non-binary                 103
Prefer not to say           33
Other (please specify)      10
Name: count, dtype: int64

In [31]:
# 소득수준 확인하기
df["Household Income"].value_counts()

Household Income
$150,000 or more       1336
$100,000 - $149,999     803
$75,000 - $99,999       427
$50,000 - $74,999       323
$25,000 - $49,999       260
Under $25,000           144
Name: count, dtype: int64

## 결측치 드랍 및 새롭게 데이터프레임 만들기

In [36]:
# 결측치 드랍하기
df= df.dropna(subset = ["What is your age?", "What is your favorite coffee drink?", "Gender", "Household Income"])

In [39]:
# 사용할 컬럼들만 모아서 새롭게 데이터프레임 만들기
df_selected = df[["What is your age?", "What is your favorite coffee drink?", "Gender", "Household Income"]]

In [43]:
df_selected.head()

,What is your age?,What is your favorite coffee drink?,Gender,Household Income
15,<18 years old,Pourover,Other (please specify),"Under $25,000"
17,25-34 years old,Regular drip coffee,Female,"$150,000 or more"
34,45-54 years old,Regular drip coffee,Male,"$100,000 - $149,999"
41,25-34 years old,Pourover,Male,"Under $25,000"
42,35-44 years old,Iced coffee,Male,"$75,000 - $99,999"


In [53]:
# Power BI에 사용할 SCV파일로 저장하기
df_selected.to_csv("coffee_data.csv", index = False)

- Pourover는 '핸드'드립 커피(=사람이 직접 붓는)
- Regular drip coffee는 커피 머신을 사용

# Power BI를 활용한 시각화 및 데이터 해석

<img src = "./image/coffee_percentage.png">

<img src = "./image/coffee_man.png" >

- 약 72%의 성별이 남성임을 통해 각 그래프 마다 남성의 비중이 매우 높음을 알 수 있음
- 2번째 사진은 남성에 해당하는 나머지 컬럼의 비율임

<img src = "./image/tree_coffee_man.png">

- 이 트리형식 그래프를 통해 Pourover(핸드드립 커피)는 대부분 남성이 먹음을 알 수 있음
  - 게다가 그들의 소득 수준은 연봉 약 2억원이 50% 이상임

<img src = "./image/tree_coffee_women.png">

- 72%가 남성임에도 불구하고 라떼의 경우는 남녀의 비율이 거의 동일함
  - 즉, 커피는 라떼류가 성별 무관하게 가장 잘 나가는 메뉴임을 확인할 수 있음